##### Loading libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#from openai import OpenAI

from jobsearch.database import fetch_data_from_postgresql
from jobsearch.params import OPENAI_API_KEY

from jsonschema import validate
import json


pd.options.display.max_colwidth = 10000
pd.options.display.max_rows = 200

##### Loading data

In [2]:
df = fetch_data_from_postgresql(columns='id, description', import_from_cloud=True)
ft_df = pd.read_csv('../db/llm_fine-tuning_dataset.csv')

#df_all = pd.read_csv('../db/jobs_02-02-2024.csv')
#df = pd.read_csv('../db/all_french_descriptions_08-02-2024.csv')
# with open('../db/json_finetuning_data.json') as f:
#     json_data = json.load(f)

Using CLOUD postgres database ...


##### quick prepro

In [31]:
taken_ids = ft_df.id.tolist()
df_ = df[~df['description'].isin(taken_ids)]
df_ = df_[~df_.description.isna()]
df2 = df_[df_.description.str.contains('de') & (df_.description.str.contains('à partir de')) & (df_.description.str.contains('€'))]#[5:100]

In [39]:
from jobsearch.cleaning import convert_to_tfidf, compute_cosine_similarity, convert_cosine_similarity_to_coo, filter_over_threshold

In [38]:
df2_list = df2.description.tolist()
df2_tfidf = convert_to_tfidf(df2_list)
df2_cosine_matrix = compute_cosine_similarity(df2_tfidf)
cosine_coo = convert_cosine_similarity_to_coo(df2_cosine_matrix)

In [1]:
overth = filter_over_threshold(df2_cosine_matrix)

idx_infer_2 = [idx for idx, over_th_array in enumerate(overth.values()) if len(overth[idx]) == 0]
#df[df.index.isin(idx_infer_2)]

NameError: name 'filter_over_threshold' is not defined

### Craft fine-tuning dataset

##### Get job description length distribution

In [ ]:
descriptions = df.description
descriptions_len = descriptions.str.len()

descriptions_len.hist(bins=200)
plt.xlim([0,10000])

descriptions_len.describe().round()

#### rough discrimination of descriptions with infos related to salaries

In [ ]:
# get descriptions ids w/ high probability of wage info in it
ids1 = df[df.description.str.lower().str.contains(r'remuneration', regex=True)].id.to_list()
ids2 = df[df.description.str.lower().str.contains(r'salaire', regex=True)].id.to_list()
ids3 = df[df.description.str.lower().str.contains(r'euros', regex=True)].id.to_list()
ids4 = df[df.description.str.lower().str.contains(r'k€|,00€|€ par', regex=True)].id.to_list()
wage_ids = set(ids1 + ids2 + ids3 + ids4)
len(wage_ids)

#### sampling descriptions for fine-tuning

In [ ]:
salaries_init = df[df.id.isin(wage_ids)]
others = df[~df.id.isin(wage_ids)]

sample1 = salaries_init.sample(90, random_state=42)
sample2 = others.sample(20, random_state=42)


In [ ]:
print(df[df.description.str.contains('tjm')].shape)

#df[df.description.str.contains(r'(\d+\/\d+)\s{0,3}€', regex=True)]
df[df.description.str.contains(r'\d+-\d+\s{0,3}€', regex=True)]
df[df.description.str.contains(r'\d{0,4}k€', regex=True)]

#### get handipcked ids with salaries info in it

In [ ]:
with open('../db/handpicked_ids_high_quality_job_descrptions_for_fine-tuning.txt', 'r') as fichier:
    handpicked_ids = fichier.read().split(', ')
    handpicked_ids = set([int(ids_str) for ids_str in handpicked_ids])

In [ ]:
sample = df[df.id.isin(handpicked_ids)]

In [ ]:
handpicked_df

#### concat all records chosed for fine-tuning

In [ ]:
sample = pd.concat([sample1, sample2, handpicked_df], axis=0)

# remove ids used as examples in prompt
sample = sample[~sample.id.isin([34, 170, 1914])]

#### language detection second pass

In [ ]:
from jobsearch.ml_logic.lang_classifier import detect_language

lang = detect_language(model_path='../models/xlm-roberta-base-language-detection/', df=sample)

In [ ]:

#sample = sample.drop('lang', axis=1)
sample['new_lang'] = lang.tolist()

# keep french records only
sample = sample[(sample.new_lang == 'fr') & (sample.lang == 'fr')]

#### Keep descriptions of specific length only

In [ ]:
lower_bound=descriptions_len.describe().iloc[4]
upper_bound=descriptions_len.describe().iloc[6]

df = df[(descriptions.str.len() >= lower_bound) & (descriptions.str.len() >= upper_bound)]

In [ ]:
df[df.description.str.contains('k€')]

In [ ]:
df = df[['id', 'description']]
n = 100
df[n:n+100].head(2)

### Prompt design 

##### prompt salaire, complements, stack, langues, etude, remote ...

In [ ]:
# all_instructions = """Tu dois extraire les informations clés d'une offre d'emploi au format JSON :

# 1. Salaire:
# - Pour un salaire indiqué comme minimum (ex. 'à partir de'), utilise-le pour le champ 'montant_min'.
# - Pour un salaire indiqué comme maximum (ex. 'jusqu'à'), saisis-le dans 'montant_max'.
# - En cas de montant unique sans autres indications, remplis à la fois 'montant_min' et 'montant_max' avec cette valeur.

# format : dict

# 2. Compléments de salaire:
# - Dans le cas où l'offre mentionne explicitement des éléments additionnels au salaire de base, comme des primes ou des avantages, liste ces éléments dans le champ 'avantages_complementaires'.
# - Inclure uniquement les éléments qui décrivent le type d'avantage offert, sans détails supplémentaires sur les conditions ou les pourcentages. Par exemple, 'Carte ticket restaurant' suffit sans mentionner le pourcentage de prise en charge.

# format : list

# 2. Stack Technologique:
# - Si l'offre d'emploi spécifie certaines technologies, outils, librairies ou langages de programmation (ex. 'Recherche développeur Java, expérience en SQL souhaitée'), liste ces technologies dans le champ 'stack_technologique'.

# format : list

# 4. Niveau d'Étude:
# - Note le niveau d'étude demandé dans le champ 'niveau_etude'. Par exemple, si l'offre mentionne 'Bac+5 requis', ajoute 'Bac+5' dans le champ correspondant.

# format : str

# 5. Langues Parlées:
# - Liste les langues parlées demandées sous 'langues_parlees'. Par exemple, si l'offre indique 'Anglais courant nécessaire', ajoute 'Anglais' dans le champ correspondant.

# format : list

# 6. Travail en distanciel / remote:
# - Indique dans le champs 'remote' si l'offre propose du télétravail par true ou false. 

# format : bool

# 7. Informations Manquantes:
# - En cas d'absence d'information, utilise null pour les champs numériques comme 'montant_min' et 'montant_max'.
#         - une liste vide pour les champs de type liste comme 'avantages_complementaires', 'niveau_etude' et 'langues_parlees'.
#         - reste le plus proche possible du verbatim et limite les interprétations.

# """

In [ ]:
# examples = """

# Chez Foxintelligence, nous sommes des amoureux de la data.\n\nNotre mission est de rendre les consommateurs et les marques plus intelligents, grâce à une donnée de marché révolutionnaire issue de l'intelligence collective. Pour les consommateurs, Foxintelligence rend accessibles et utiles leurs données transactionnelles issues de leurs boîtes mail. En les anonymisant, nous créons des statistiques uniques qui permettent aux marques de rester en phase avec les produits qui se vendent le mieux et les attentes des consommateurs.\n\nNous publions une application grand public aimée par des millions de personnes (Cleanfox, 4m+ de téléchargements, 4.8/5 sur l'app store et le play store) et préparons le lancement d'une app de bilan carbone à l'expérience radicalement nouvelle. Grâce à notre plateforme data, elle va rendre possible une prise de conscience massive des enjeux du changement climatique et des actions individuelles possibles pour le combattre. Nous pensons que l'information est une... force de changement puissante, lorsqu'elle s'appuie sur la data.\n\nUtilisant ces données personnelles totalement anonymisées, notre plateforme SaaS est devenue la référence de la market intelligence digitale en Europe avec des dizaines de grands groupes clients. Le point commun entre Deliveroo, Just Eat, BackMarket ou The Boston Consulting Group ? Ils vont très vite (on ne s'ennuie pas avec eux ) et ils nous font tous confiance\n\nSoutenus par des investisseurs de premier plan et récemment intégrés au groupe NielsenIQ (17m levés auprès de Daphni, Partech, GFC et eFounders), nous sommes maintenant à la conquête de l'Europe et en phase d'hyper-croissance (+100% attendue en 2022).\n\nSi notre FoxHQ reste à Paris, notre équipe de plus de 100 talents pluridisciplinaires (tech, data et business) travaille depuis partout en France. Nous pensons que notre politique de remote, notre culture forte (bienveillance, exigence, résilience et data-first) et notre innovation permanente en termes de modes de travail (ex. grille de salaire transparente, formation au changement climatique, transparence sur la stratégie) sont les clés de notre réussite collective.... et de ta réussite avec nous\nJob Description\n\nFoxintelligence recherche un(e) stagiaire Data Analyst.\n\nTu seras rattaché(e) à l'équipe Data Management et tu interviendras au niveau de l'enrichissement de la donnée. Cet enrichissement rend possible l'analyse de la donnée par nos équipes analytics et nos clients. Ton rôle sera de garantir que les champs enrichis aient le niveau de completion et de qualité attendues sur l'ensemble de nos industries (e-commerce, food delivery etc.) et territoires couverts (7 pays européens, ainsi que l'Inde).\n\nTes responsabilités au quotidien :\n• Qualité et implémentation de règles métier : tu auras une roadmap de marchands à enrichir, tu seras garant(e) de la qualité des enrichissements et de leur disponibilité à la date attendue. Tu travailleras main dans la main avec différentes équipes en interne (équipes clients, data management) pour y arriver.\n• Amelioration et optimisation continue des outils\n• Investigation sur les sujets liés à l'enrichissement : tu devras être capable de comprendre les problématiques liés aux enrichissements pour répondre aux besoins clients.\n\nRequirements\n\nCompétences nécessaires (ce qu'il te faut pour réussir sur ce poste) :\n\nSoft skills\n• Esprit analytique et structuré\n• Tu as la volonté d'apprendre, de contribuer spontanément à ton environnement et d'aider l'équipe à s'améliorer\n• Tu aimes découper et optimiser des process, pour être toujours plus efficace\n\nHard skills\n• Gestion de projet : tu sais t'organiser, gérer efficacement ton temps de travail, et communiquer avec des interlocuteurs divers pour faire avancer ton projet\n• Tu es as l'aise en SQL\n\nCompétences bonus\n• Maitrise de Tableau, Github, Regex\n• Tu parles couramment allemand, espagnol ou italien ou néerlandais\n• Tu as une première expérience en gestion de projet\n\nLadies\n\nLes études montrent que les femmes ont moins tendance à postuler à une offre d'emploi quand elles n'ont pas toutes les qualifications requises. Ladies, ne vous mettez pas de barrière et donnez-nous la chance de nous faire notre propre avis, nous serons toujours ravis d'échanger avec vous Si notre raison d'être vous parle, postulez\n\nSelf-made data lovers\n\nLes diplômes c'est bien, les skills c'est mieux et l'expérience t'en donne. Aucun diplôme n'est requis chez nous, c'est les compétences et l'énergie qui comptent\nRecruitment process\n• Round 0 : entretien (fit) avec un membre notre équipe HR\n• Round 1 : entretien (expérience et fit) avec un(e) Senior Data Analyst de l'équipe\n• Round 2 : test technique\n• Decision Round : entretien avec notre Lead Data Analyst\n\nBenefits\n\nAvantages/ce que nous offrons\n• Rémunération Compétitive\n• Remote friendly (+ budget aménagement de l'espace de travail @Home)\n• Carte ticket restaurants (Swile) pris en charge à 60% par Fox\n• Culture forte et pratiques de management à la pointe (stratégie et résultats financiers transparents, feedback 360, grille de salaire innovante et transparente etc

# {
#   "salaire": {
#     "montant_min": null,
#     "montant_max": null
#   },
#   "avantages_complementaires": [
#     "Remote friendly",
#     "Carte ticket restaurants"
#   ],
#   "stack_technologique": [
#     "SQL",
#     "Tableau",
#     "Github",
#     "Regex"
#   ],
#   "niveau_etude": null,
#   "langues_parlees": ["Allemand", "Espagnol", "Italien", "Néerlandais"],
#   "remote": true
# }

# """

##### salary prompt + prompt token count

In [1]:
# load prompt
with open("../db/salary_extraction_prompt-v2.txt") as f:
    prompt = f.read()

In [ ]:
# word_count = len(prompt.split(' '))
# token_count = round(word_count * 1.4) # 1.4 = approximation of french word/token conversion factor

# print('Word count : ', word_count)
# print('Token count : ', token_count)

# print('\n================== PROMPT ==================\n', '\n', prompt)

##### training data token count

In [ ]:
# df = df[~df.isna()]
# token_count = df.description.str.split(' ').str.len() * 1.4

In [ ]:
# bound1 = token_count.describe(percentiles=np.arange(0.99,1, 0.001))#[6]
# df[token_count > bound1]

### Request openai API

In [6]:
df_.description[df_.description.str.contains('remunération')]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [3]:
def request_openai_json_output(prompt_list):
  client = OpenAI(api_key=OPENAI_API_KEY)
  json_outputs = []

  for description in prompt_list:
    # models
    GPT35 = "gpt-3.5-turbo-1106"
    GPT4 = "gpt-4-0613" # doesn't support json_object response format type 
    GPT4_32K = "gpt-4-32k-0613"

    response = client.chat.completions.create(
      model=GPT35,
      response_format={ "type": "json_object" },
      messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": description}
      ],
    )
    json_response = response.choices[0].message.content
    json_outputs.append(json_response)
    
    print(json_response)
    return json_outputs

In [ ]:
json_outputs = request_openai_json_output()

### Post processing outputs

##### JSON outputs validity

In [ ]:
def check_extracted_salaries_json_validity(output_list):

  json_validity = []
  for json_response in output_list:
    schema = {
      "type" : "object",
      "properties" : {
        "salaire" : {
          "type" : "array",
          "items" : {
            "type" : "object",
            "properties" : {
              "montant_min" : {"type" : ["number", "null"]},
              "montant_max" : {"type" : ["number", "null"]},
              "devise" : {"type" : ["string", "null"]},
              "frequence_versement" : {"type" : ["string", "null"]}
            },
            "required": ["montant_min", "montant_max", "devise", "frequence_versement"]
          }
        }
      },
      "required" : ["salaire"]
    }

    try:
        validate(instance=json.loads(json_response), schema=schema)
        json_validity.append(True)
        #print(True)
    except Exception as e:
        json_validity.append(e)
        #print(f"Le JSON est invalide : {e}")
        
    return json_outputs_validity
  
  
json_outputs_validity = check_extracted_salaries_json_validity(json_outputs)

##### Concat ids + outputs + descriptions

In [ ]:
df_outputs = pd.DataFrame(zip(sample.id, sample.description, json_outputs, json_outputs_validity), columns=["id", "description", "gpt_output", "json_validity"])

# convert str output to json output
df_outputs.gpt_output = df_outputs.gpt_output.map(lambda x: json.loads(x))

# display invalid json outputs
df_outputs[df_outputs.json_validity != True]

##### Convert string to valid json string

In [ ]:
import re

def convert_to_valid_json_string(input_string):
    # Replace property names: Look for anything that matches 'word': (including nested properties)
    # and replace with "word":
    pattern_props = r"(\s*)'(\w+)'\s*:"
    subst_props = r'\1"\2":'
    result = re.sub(pattern_props, subst_props, input_string)
    
    # Replace string values: Look for ':' followed by a single-quoted string
    # This is a simplistic approach and might need adjustment based on your data's complexity
    pattern_values = r":\s*'([^']*)'"
    subst_values = r': "\1"'
    result = re.sub(pattern_values, subst_values, result)

    # Handle None values to be JSON compliant (convert None to null)
    result = result.replace(": None", ": null")

    return result


df['tojson'] = df['gpt_output'].map(convert_to_valid_json_string)

##### Convert to valid json/dict

In [ ]:
df.tojson = df['tojson'].map(json.loads)

##### Add metadata key to json file

In [ ]:
complete_jsons = []
for id, dicos in zip(df.id, df.tojson):
    complete_json = {"metadata": {"id": id}, "data": dicos}
    complete_jsons.append(complete_json)
    
df['complete_json'] = complete_jsons

In [ ]:
output_list = df['complete_json'].tolist()

##### Convert data to alpaca dataset format

In [ ]:
with open("../db/json_finetuning_data_alpaca_format.jsonl", "w") as f:
    for n in enumerate(json_data):
        
        # Extract 'input' based on 'id' from 'df'
        input_text = df.description[df.id == json_data[n[0]]['metadata']['id']].values[0]
        output = json_data[n[0]]['data']
        #output_json_string = json.dumps(output, ensure_ascii=False)
        #print(type(output_json_string))
        # Construct the object in the desired format
        alpaca_format = {
            "instruction": prompt[0:10],  # Assuming 'prompt' is defined elsewhere
            "input": input_text[0:10],
            "output": str(output).replace("'", '"')
        }
        
        # Convert the object to a JSON string and append a newline
        jsonl_string = json.dumps(alpaca_format, ensure_ascii=False) + '\n'
        
        # Write the JSON string directly to the file
        f.write(jsonl_string)

In [ ]:
# df[['complete_json']].to_json('../db/json_finetuning_data.json', orient='records')

### Export data

##### to local JSONL file

In [ ]:
with open("../db/json_finetuning_data_alpaca_format.jsonl", "w") as f:
    for n in enumerate(json_data):
        
        # Extract 'input' based on 'id' from 'df'
        input_text = df.description[df.id == json_data[n[0]]['metadata']['id']].values[0]
        output = json_data[n[0]]['data']
        output_json_string = json.dumps(output, ensure_ascii=False)

        # Correctly serialize 'output' to a JSON-formatted string
        #output_json_string = json.dumps(output, ensure_ascii=False)
        
        # Construct the object in the desired format with 'output' as a JSON string
        alpaca_format = {
            
            "instruction": prompt,  # Truncating prompt for brevity
            "input": input_text,    # Truncating input for brevity
            "output": output_json_string #str(output).replace("'", '"')
        }
        
        # Serialize the entire object to a JSON string and append a newline
        jsonl_string = json.dumps(alpaca_format, ensure_ascii=False) + '\n'
        
        # Write the JSON string directly to the file
        f.write(jsonl_string)


##### to local JSON file

In [ ]:
with open('../db/json_finetuning_data.json', 'w') as f:
    json.dump(output_list, f, indent=None)

##### to hugging face hub

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# read and split
tiger_llama =  pd.read_json('../db/json_finetuning_data_alpaca_format.jsonl', lines=True)
train_tiger, test_tiger = train_test_split(
    tiger_llama, test_size=0.15, random_state=42, shuffle=True
)

# push to hub
train_tiger = Dataset.from_pandas(train_tiger)
test_tiger = Dataset.from_pandas(test_tiger)

ds = DatasetDict()
ds["train"] = train_tiger
ds["test"] = test_tiger

dataset_name = "axel-rda/tech_job_salaries_prediction_fr"
ds.push_to_hub(dataset_name, branch="main", private=True)